In [6]:
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np

model =Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

TRAINING_DIR = "/Users/mohan/Downloads/face-mask-detector-project/Dataset/test"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
VALIDATION_DIR = "/Users/mohan/Downloads/face-mask-detector-project/Dataset/test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')


history = model.fit_generator(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

Found 194 images belonging to 2 classes.
Found 194 images belonging to 2 classes.
Epoch 1/10
20/20 [==============================] - 41s 2s/step - loss: 1.0070 - acc: 0.4794 - val_loss: 0.6958 - val_acc: 0.5155
Epoch 2/10
20/20 [==============================] - 39s 2s/step - loss: 0.6926 - acc: 0.4897 - val_loss: 0.7011 - val_acc: 0.5052
Epoch 3/10
20/20 [==============================] - 40s 2s/step - loss: 0.7090 - acc: 0.5361 - val_loss: 0.6945 - val_acc: 0.5825
Epoch 4/10
20/20 [==============================] - 42s 2s/step - loss: 0.6927 - acc: 0.5000 - val_loss: 0.6889 - val_acc: 0.6495
Epoch 5/10
20/20 [==============================] - 42s 2s/step - loss: 0.6957 - acc: 0.5825 - val_loss: 0.4630 - val_acc: 0.7938
Epoch 6/10
20/20 [==============================] - 43s 2s/step - loss: 0.5775 - acc: 0.7577 - val_loss: 1.1647 - val_acc: 0.7835
Epoch 7/10
20/20 [==============================] - 43s 2s/step - loss: 0.5139 - acc: 0.7887 - val_loss: 0.0774 - val_acc: 0.7990
Epoch 8/

In [2]:
pip install imutils

  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25860 sha256=329f334edb8e951e95f0db9ae9e80f09a554dde7922e3fec652bef36ae27b02e
  Stored in directory: /Users/mohan/Library/Caches/pip/wheels/86/d7/0a/4923351ed1cec5d5e24c1eaf8905567b02a0343b24aa873df2
Successfully built imutils
Note: you may need to restart the kernel to use updated packages.


In [11]:
import cv2
import numpy as np
from keras.models import load_model
model=load_model("/Users/mohan/Downloads/face-mask-detector-project/model2-005.model")
results={0:'without mask',1:'mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}
rect_size = 4
cap = cv2.VideoCapture(0) 
haarcascade = cv2.CascadeClassifier('/Users/mohan/Downloads/haarcascade_frontalface_default.xml')
while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 
    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    
    if key == 27: 
        break
cap.release()
cv2.destroyAllWindows()